---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [7]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [277]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni_towns = pd.read_csv('university_towns.txt',sep="\n", header=None, names=['RegionName'])
    
    # Enter state name in column 'State' in the same row as 'RegionName'. Set everything else to NaN.
    uni_towns['State'] = np.where(uni_towns['RegionName'].str.contains('edit'),uni_towns['RegionName'],np.NaN)
        
    uni_towns['State'].fillna(method='ffill',inplace=True) # Forward fill state names to replace NaN values.
    
    uni_towns = uni_towns[['State', 'RegionName']] # rearrange data 
    
    # remove extra [] and () from both columns 
    
    for col in uni_towns:
        uni_towns[col] = uni_towns[col].str.split('[', n=1, expand=True)[0]
        uni_towns[col] = uni_towns[col].str.split('(', n=1, expand=True)[0].str.rstrip()
    
    uni_towns = uni_towns[uni_towns['RegionName'] != uni_towns['State']] #remove state rows

    return  uni_towns 

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [125]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    rec_start = pd.read_excel('gdplev.xls', skiprows = 7, skipheader = 5)
    
    rec_start.columns = ['Year', 'GDP Current:Annual', 'GDP 2009:Annual', 'Blank 1', 'YearQ', 'GDP Current:Q', 'GDP 2009:Q', 'Blank 2']
    
    rec_start = rec_start.drop(['Blank 1', 'Blank 2'], axis=1)
    
   
    
    rec_start['RecessionQ']  = rec_start['GDP 2009:Q'].diff()
    
    rec_start = rec_start[rec_start['YearQ'].str.contains('^2')] #remove state rows
    
    rec_start = rec_start.drop(['Year','GDP Current:Annual', 'GDP 2009:Annual', 'GDP Current:Q'], axis = 1)
    
     # rec_start : Recession start is defined by starting with two consecutive quarters of GDP decline
    
    cond = (rec_start['RecessionQ'] < 0) & (rec_start['RecessionQ'].shift(-1) <0) & (rec_start['RecessionQ'].shift(1) > 0)
    
    rec_start1 = rec_start[cond]
    
        #(rec_start['RecessionQ'].shift(1) >0) & 
        #cond1=(df['Change'].shift(1)>0) & (df['Change']<0) & (df['Change'].shift(-1)<0)
    return rec_start1['YearQ'].item()

get_recession_start()

'2008q3'

In [214]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # ending with two consecutive quarters of GDP growth
    
    rec_start = pd.read_excel('gdplev.xls', skiprows = 7, skipheader = 5)
    
    rec_start.columns = ['Year', 'GDP Current:Annual', 'GDP 2009:Annual', 'Blank 1', 'YearQ', 'GDP Current:Q', 'GDP 2009:Q', 'Blank 2']
    
    rec_start = rec_start.drop(['Blank 1', 'Blank 2'], axis=1)
    
    rec_start['RecessionQ']  = rec_start['GDP 2009:Q'].diff()
    
    rec_start = rec_start[rec_start['YearQ'].str.contains('^2')] #remove state rows
    
    rec_start = rec_start.drop(['Year','GDP Current:Annual', 'GDP 2009:Annual', 'GDP Current:Q'], axis = 1)
    

    cond = (rec_start['RecessionQ'] > 0) & (rec_start['RecessionQ'].shift(1) > 0) & (rec_start['RecessionQ'].shift(2) < 0)
    
    #rec_end = rec_start[cond]
    
    start = get_recession_start()
    
    index_start = rec_start.index[rec_start['YearQ'] == start].values.astype(int)[0]
    
    #df.index[df['BoolCol'] == True].tolist()
    
    rec_s = rec_start.loc[index_start:]
    
    rec_end = rec_s[cond]

    rec_end = rec_end.iloc[0]
       
    return rec_end['YearQ']

get_recession_end()

'2009q4'

In [199]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    rec_start = pd.read_excel('gdplev.xls', skiprows = 7, skipheader = 5)
    
    rec_start.columns = ['Year', 'GDP Current:Annual', 'GDP 2009:Annual', 'Blank 1', 'YearQ', 'GDP Current:Q', 'GDP 2009:Q', 'Blank 2']
    
    rec_start = rec_start.drop(['Blank 1', 'Blank 2'], axis=1)
    
    rec_start['RecessionQ']  = rec_start['GDP 2009:Q'].diff()
    
    rec_start = rec_start[rec_start['YearQ'].str.contains('^2')] #remove state rows
    
    rec_start = rec_start.drop(['Year','GDP Current:Annual', 'GDP 2009:Annual', 'GDP Current:Q'], axis = 1)
    

   # cond = (rec_start['RecessionQ'] > 0) & (rec_start['RecessionQ'].shift(1) > 0) & (rec_start['RecessionQ'].shift(2) < 0)
    
    #rec_end = rec_start[cond]
    
    start = get_recession_start()
    
    index_start = rec_start.index[rec_start['YearQ'] == start].values.astype(int)[0]
    
    #index_start = index_start.get_loc
    
    #index_start = 246
    
    end = 251
    
    rec_bottom = rec_start.loc[(index_start) : (end)]
    
    
    rec_bottom = rec_bottom[rec_bottom['GDP 2009:Q'] == rec_bottom['GDP 2009:Q'].min()]
    
    return rec_bottom['YearQ'].item()

get_recession_bottom()

'2009q2'

In [250]:

# Change year and month column header to year and quarter:

def change_to_quarter(date: str):
    date = date.split('-')
    month = int(date[1])
    quarter = int((month - 1) / 3) + 1
    return date[0] + 'q' + str(quarter)

In [268]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    h_data_org = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # A quarter is a specific three month period,
    #Q1 is January through March, -03
    #Q2 is April through June,  -06
    #Q3 is July through September, -09
    #Q4 is October through December. -12
    
    h_data = h_data_org.loc[:, (h_data_org.columns.str.contains('^1') == False)] #remove columns before 2000q1
    
    
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-01') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-02') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-04') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-05') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-07') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-08') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-10') == False)]
   # h_data = h_data.loc[:,(h_data.columns.str.contains('-11') == False)]
    
   # h_data.columns = [sub.replace('-03', 'q1') for sub in h_data.columns] 
   # h_data.columns = [sub.replace('-06', 'q2') for sub in h_data.columns] 
   # h_data.columns = [sub.replace('-09', 'q3') for sub in h_data.columns] 
   # h_data.columns = [sub.replace('-12', 'q4') for sub in h_data.columns] 
    
    h_data = h_data.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
    #h_data = h_data.drop(h_data.columns[[0]], axis=0)
    
    h_data = h_data.replace({'State': states})
    
    h_data = h_data.set_index(['State','RegionName']).sort_index()
    
    h_data = h_data.groupby(change_to_quarter, axis=1).mean() # Find mean over the months in a quarter.
    
    
    return h_data
#convert_housing_data_to_quarters()
#convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]

2010q3        2015q2  2016q4
State RegionName                                    
Ohio  Akron       67566.666667  61833.333333     NaN
      Dayton      57600.000000  50000.000000     NaN

In [278]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    
    # Start position is the quarter BEFORE the recession starts!
    before_rec = (df.columns.get_loc(get_recession_start())-1)
    rec_bottom = df.columns.get_loc(get_recession_bottom())
    
    uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # Turn the divided values into a DataFrame!
    df = np.divide(df.ix[:,before_rec],df.ix[:,rec_bottom]).to_frame().dropna()
    
    #df_uni_merge = df.merge(uni, left_index = True, right_index = True, how='inner')
    uni_df = df.merge(uni, right_index=True, left_index=True, how='inner')
    
    # Drop the indices of uni towns to get data only for non uni towns.
    nonuni_df = df.drop(uni_df.index)
    
    # A t-test is commonly used to determine whether the mean of a population significantly
    # differs from a specific value (called the hypothesized mean) or from the mean of another population.
    p_value = ttest_ind(uni_df.values, nonuni_df.values).pvalue
    
    if p_value < 0.01:
        different=True
    else:
        different=False
        
    # Better depending on which one is LOWER! Remember prices go up during a recession so lower is better.
    if uni_df.mean().values < nonuni_df.mean().values:
        better='university town'
    else:
        better='non-university town'
    
    return (different, p_value[0], better)
run_ttest()

(True, 0.002724063704761164, 'university town')